<a href="https://colab.research.google.com/github/ldebris/dm2025a/blob/main/Arboles_Canaritos_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/dm"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dm" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/itba2025-8d0a/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi

In [14]:
if (!require("rpart.plot")) install.packages("rpart.plot")
library(rpart.plot)

# librerías necesarias
library(data.table)
library(rpart)
library(rpart.plot)

# parámetros
semillas <- 100001 + 1:20  # 20 semillas diferentes
cp_prunings <- c(-666, -777, -888)  # tipos de poda

# cargo dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# defino train y test
dtrain <- dataset[foto_mes == 202107]
dtrain[, clase_binaria2 := ifelse(clase_ternaria == "CONTINUA", "NEG", "POS")]
dtrain[, clase_ternaria := NULL]

dfuture <- dataset[foto_mes == 202109]


In [15]:
entrenar_y_prunear <- function(semilla, dtrain, dfuture, PARAM, n_canaritos = 300, pesos_pos = 5.0) {
  set.seed(semilla)

  # Agregar canaritos a entrenamiento
  datos_train <- copy(dtrain)
  for (i in 1:n_canaritos) {
    datos_train[, paste0("canarito", i) := runif(.N)]
  }

  # Agregar canaritos a scoring (dfuture)
  datos_future <- copy(dfuture)
  for (i in 1:n_canaritos) {
    datos_future[, paste0("canarito", i) := runif(.N)]
  }

  # Definir pesos
  pesos <- ifelse(datos_train$clase_binaria2 == "POS", pesos_pos, 1.0)

  # Entrenar modelo sin poda
  modelo_original <- rpart::rpart(
    formula = clase_binaria2 ~ .,
    data = datos_train,
    model = TRUE,
    xval = 0,
    control = PARAM$rpart,
    weights = pesos
  )

  # Poda para diferentes valores de cp
  cp_values <- c(-666, -777, -888)
  preds_list <- list()

  for (cp_val in cp_values) {
    modelo_original$frame[modelo_original$frame$var %like% "canarito", "complexity"] <- cp_val
    modelo_podado <- prune(modelo_original, cp_val)

    pred <- predict(modelo_podado, datos_future, type = "prob")[, "POS"]

    nombre_col <- paste0("pred_s", semilla, "_cp", cp_val)
    preds_list[[nombre_col]] <- pred
  }

  return(preds_list)
}




In [16]:
PARAM <- list()
PARAM$rpart <- list(
  cp = -1,
  minsplit = 3800,
  minbucket = 48,
  maxdepth = 30
)


In [ ]:

# PARAM <- list()
# PARAM$rpart <- list(
#   cp = -1,
#   minsplit = 3800,
#   minbucket = 48,
#   maxdepth = 30
# )

# Definición corregida de la función entrenar_y_prunear
entrenar_y_prunear <- function(semilla, dtrain, dfuture, PARAM, n_canaritos = 300, pesos_pos = 5.0) {
  set.seed(semilla)

  datos_train <- copy(dtrain)
  for (i in 1:n_canaritos) {
    datos_train[, paste0("canarito", i) := runif(.N)]
  }

  datos_future <- copy(dfuture)
  for (i in 1:n_canaritos) {
    datos_future[, paste0("canarito", i) := runif(.N)]
  }

  pesos <- ifelse(datos_train$clase_binaria2 == "POS", pesos_pos, 1.0)

  modelo_original <- rpart::rpart(
    formula = clase_binaria2 ~ .,
    data = datos_train,
    model = TRUE,
    xval = 0,
    control = PARAM$rpart,
    weights = pesos
  )

  cp_values <- c(-666, -777, -888)
  preds_list <- list()

  for (cp_val in cp_values) {
    modelo_original$frame[modelo_original$frame$var %like% "canarito", "complexity"] <- cp_val
    modelo_podado <- prune(modelo_original, cp_val)

    pred <- predict(modelo_podado, datos_future, type = "prob")[, "POS"]

    nombre_col <- paste0("pred_s", semilla, "_cp", cp_val)
    preds_list[[nombre_col]] <- pred
  }

  return(preds_list)
}

# Ahora el loop corregido que llama a la función pasando PARAM:
preds_total <- list()

for (sem in semillas) {
  cat("Entrenando semilla:", sem, "\n")
  preds <- entrenar_y_prunear(sem, dtrain, dfuture, PARAM, n_canaritos = 300, pesos_pos = 5.0)
  preds_total <- c(preds_total, preds)  # acumulo todas las columnas
}

# Creo data.table final de probabilidades
tb_ensemble <- data.table(numero_de_cliente = dfuture$numero_de_cliente)

# Agrego columnas de cada modelo
for (nombre_col in names(preds_total)) {
  tb_ensemble[, (nombre_col) := preds_total[[nombre_col]] ]
}

# Calculo promedio
tb_ensemble[, prob := rowMeans(.SD), .SDcols = names(preds_total)]

# Decido top 11000
setorder(tb_ensemble, -prob)
tb_ensemble[, Predicted := 0L]
tb_ensemble[1:11000, Predicted := 1L]

# Exporto a Kaggle
fwrite(tb_ensemble[, .(numero_de_cliente, Predicted)],
       file = "K_super_ensemble.csv", sep = ",")

# Envío a Kaggle
comando <- paste(
  "kaggle competitions submit",
  "-c data-mining-analista-sr-2025-a",
  "-f K_super_ensemble.csv",
  "-m 'Super ensemble 60 árboles: 20 seeds x 3 podas (-666/-777/-888)'"
)
cat("Ejecutando:\n", comando, "\n")
salida <- system(comando, intern=TRUE)
cat(salida, "\n")

